# Compare CM and MC
Comparing contrast-meta and meta-contrast

In [ ]:
import os
import sys
import os
import numpy as np
import pandas as pd
import nibabel as nib

from similarity import calculate_clus_similarity, calculate_peak_similarity, calculate_voxel_similarity

In [ ]:
# input paths
gm_mask = os.path.abspath('../pyALE/utils/mask/Grey10.nii')

# output paths
output_dir_main = os.path.abspath('../../docs/manuscript/Rmd/tabs/')
output_dir_supplement = os.path.abspath('../../docs/manuscript/Rmd/tabs/suppl/')


**1. Voxel comparison:** How many voxel overlap between both maps

**2. Peak comparison:** Median distance of peaks to next peaks
    
**3. Cluster comparison:** How many cluster in CM/MC, and overlapping with results of MC/CM.

In [ ]:
tasks=['n-back', 'n-back', 'stroop', 'emo-faces']
img1s=['2BKvsBASE--0BKvsBASE_P95', '2BKvs0BK--1BKvs0BK_P95', 'IvsNRB--CvsNRB_P95', 'EMOvsBASE--NEUvsBASE_P95']
img2s=['2BKvs0BK_cFWE05', '2BKvs1BK_cFWE05', 'IvsC_cFWE05', 'EMOvsNEU_cFWE05']

max_dist=None # 20
mean_or_median='median'

data = {
    'task': [],
    'task_name': [],
    'voxel_jaccard': [],
    'voxel_sensitivity': [],
    'voxel_precision': [],
    'voxel_MC': [],
    'voxel_CM': [],
    'voxel_overlap': [],
    'peak_distance_MC': [],
    'peak_distance_CM': [],
    'peak_distance_both': [],
    'cluster_CM': [],
    'cluster_overlap_CM_with_MC': [],
    'cluster_sensitivity': [],
    'cluster_MC': [],
    'cluster_overlap_MC_with_CM': [],
    'cluster_precision': []
}

print('Distance of peaks by', mean_or_median)
print('Cluster comparisons by 1 voxel overlap')
print('---------------------------')

for task, img1, img2 in zip(tasks, img1s, img2s):
    if img1 == '2BKvsBASE--0BKvsBASE_P95':
        task_name = 'Working Memory: 2-back > 0-back'
    elif img1 == '2BKvs0BK--1BKvs0BK_P95':
        task_name = 'Working Memory: 2-back > 1-back'
    elif task == 'stroop':
        task_name = 'Interference Processing'
    elif task == 'emo-faces':
        task_name = 'Emotional Face Processing'
    else:
        task_name = task
        
    # load input computed with FSL cluster
    img1_img = os.path.abspath(f'../../output/{task}/evaluation/output_clusterize/{img1}.nii.gz')
    img2_img = os.path.abspath(f'../../output/{task}/evaluation/output_clusterize/{img2}.nii.gz')
    localmax_img1 = pd.read_csv(f'../../output/{task}/evaluation/output_clusterize/{img1}_localmaxima.txt', delimiter='\t')
    localmax_img2 = pd.read_csv(f'../../output/{task}/evaluation/output_clusterize/{img2}_localmaxima.txt', delimiter='\t')
    blob_idx_img1 = os.path.abspath(f'../../output/{task}/evaluation/output_clusterize/{img1}_clusteroutput.nii.gz')
    blob_idx_img2 = os.path.abspath(f'../../output/{task}/evaluation/output_clusterize/{img2}_clusteroutput.nii.gz')

    # calculate similarity metrics
    mean_dist = calculate_peak_similarity(img1=localmax_img1, img2=localmax_img2, max_dist=max_dist, mean_or_median=mean_or_median)
    clus_comp = calculate_clus_similarity(img1=blob_idx_img1, img2=blob_idx_img2)
    vox_comp = calculate_voxel_similarity(img1=img1_img, img2=img2_img, gm_mask=gm_mask, thr_img1=1.645, thr_img2=1.645)
    
    data['task'].append(task)
    data['task_name'].append(task_name)
    data['voxel_jaccard'].append(vox_comp['jaccard'])
    data['voxel_sensitivity'].append(vox_comp['sensitivity'])
    data['voxel_precision'].append(vox_comp['precision'])
    data['voxel_MC'].append(vox_comp['voxel_map1'])
    data['voxel_CM'].append(vox_comp['voxel_map2'])
    data['voxel_overlap'].append(vox_comp['voxel_overlap'])
    data['peak_distance_MC'].append(mean_dist[0].round(2))
    data['peak_distance_CM'].append(mean_dist[1].round(2))
    data['peak_distance_both'].append(np.mean(mean_dist).round(2))
    data['cluster_CM'].append(clus_comp['num_clus2'])
    data['cluster_overlap_CM_with_MC'].append(clus_comp['olp_img2'])
    data['cluster_sensitivity'].append(round(clus_comp['olp_img2'] / clus_comp['num_clus2'], 2))
    data['cluster_MC'].append(clus_comp['num_clus1'])
    data['cluster_overlap_MC_with_CM'].append(clus_comp['olp_img1'])
    data['cluster_precision'].append(round(clus_comp['olp_img1'] / clus_comp['num_clus1'], 2))

df = pd.DataFrame(data)
df

In [ ]:
# save as csv's
df[['task_name', 'voxel_jaccard', 'voxel_sensitivity', 'voxel_precision']].to_csv(os.path.join(output_dir_main, 'comparison_scores_vox.csv'))
df[['task_name', 'peak_distance_MC', 'peak_distance_CM', 'cluster_sensitivity', 'cluster_precision']].to_csv(os.path.join(output_dir_main, 'comparison_scores_reg.csv'))


## Supplementary analysis
Peak/ cluster overlap of matched analysis

In [ ]:
tasks = ['n-back', 'n-back', 'stroop', 'emo-faces']
img1s = ['2BKvsBASE_matched_2vs0--0BKvsBASE_matched_2vs0_P95', '2BKvs0BK_matched_2vs1--1BKvs0BK_matched_2vs1_P95',
         'IvsNRB_matched--CvsNRB_matched_P95', 'EMOvsBASE_matched--NEUvsBASE_matched_P95']
img2s = ['2BKvs0BK_matched_2vs0_cFWE05', '2BKvs1BK_matched_2vs1_cFWE05', 'IvsC_matched_cFWE05', 'EMOvsNEU_matched_cFWE05']

max_dist=None # 20
mean_or_median='median'

data = {
    'task': [],
    'task_name': [],
    'voxel_jaccard': [],
    'voxel_sensitivity': [],
    'voxel_precision': [],
    'voxel_MC': [],
    'voxel_CM': [],
    'voxel_overlap': [],
    'peak_distance_MC': [],
    'peak_distance_CM': [],
    'peak_distance_both': [],
    'cluster_CM': [],
    'cluster_overlap_CM_with_MC': [],
    'cluster_sensitivity': [],
    'cluster_MC': [],
    'cluster_overlap_MC_with_CM': [],
    'cluster_precision': []
}
print('Distance of peaks by', mean_or_median)
print('Cluster comparisons by 1 voxel overlap')
print('---------------------------')

for task, img1, img2 in zip(tasks, img1s, img2s):
    if img1 == '2BKvsBASE_matched_2vs0--0BKvsBASE_matched_2vs0_P95':
        task_name = 'Working Memory: 2-back > 0-back - matched'
    elif img1 == '2BKvs0BK_matched_2vs1--1BKvs0BK_matched_2vs1_P95':
        task_name = 'Working Memory: 2-back > 1-back - matched'
    elif task == 'stroop':
        task_name = 'Interference Processing - matched'
    elif task == 'emo-faces':
        task_name = 'Emotional Face Processing - matched'
    else:
        task_name = task

    # load input computed with FSL cluster
    img1_img = os.path.abspath(f'../../output/{task}/evaluation/output_clusterize/{img1}.nii.gz')
    img2_img = os.path.abspath(f'../../output/{task}/evaluation/output_clusterize/{img2}.nii.gz')
    localmax_img1 = pd.read_csv(f'../../output/{task}/evaluation/output_clusterize/{img1}_localmaxima.txt', delimiter='\t')
    localmax_img2 = pd.read_csv(f'../../output/{task}/evaluation/output_clusterize/{img2}_localmaxima.txt', delimiter='\t')
    blob_idx_img1 = os.path.abspath(f'../../output/{task}/evaluation/output_clusterize/{img1}_clusteroutput.nii.gz')
    blob_idx_img2 = os.path.abspath(f'../../output/{task}/evaluation/output_clusterize/{img2}_clusteroutput.nii.gz')

    # calculate similarity metrics
    mean_dist = calculate_peak_similarity(img1=localmax_img1, img2=localmax_img2, max_dist=max_dist, mean_or_median=mean_or_median)
    clus_comp = calculate_clus_similarity(img1=blob_idx_img1, img2=blob_idx_img2)
    vox_comp = calculate_voxel_similarity(img1=img1_img, img2=img2_img, gm_mask=gm_mask, thr_img1=1.645, thr_img2=1.645)
    
    data['task'].append(task)
    data['task_name'].append(task_name)
    data['voxel_jaccard'].append(vox_comp['jaccard'])
    data['voxel_sensitivity'].append(vox_comp['sensitivity'])
    data['voxel_precision'].append(vox_comp['precision'])
    data['voxel_MC'].append(vox_comp['voxel_map1'])
    data['voxel_CM'].append(vox_comp['voxel_map2'])
    data['voxel_overlap'].append(vox_comp['voxel_overlap'])
    data['peak_distance_MC'].append(mean_dist[0].round(2))
    data['peak_distance_CM'].append(mean_dist[1].round(2))
    data['peak_distance_both'].append(np.mean(mean_dist).round(2))
    data['cluster_CM'].append(clus_comp['num_clus2'])
    data['cluster_overlap_CM_with_MC'].append(clus_comp['olp_img2'])
    data['cluster_sensitivity'].append(round(clus_comp['olp_img2'] / clus_comp['num_clus2'], 2))
    data['cluster_MC'].append(clus_comp['num_clus1'])
    data['cluster_overlap_MC_with_CM'].append(clus_comp['olp_img1'])
    data['cluster_precision'].append(round(clus_comp['olp_img1'] / clus_comp['num_clus1'], 2))

df = pd.DataFrame(data)
df

In [ ]:
# save as csv's
df[['task_name', 'voxel_jaccard', 'voxel_sensitivity', 'voxel_precision']].to_csv(os.path.join(output_dir_supplement, 'comparison_scores_matched_vox.csv'))
df[['task_name', 'peak_distance_MC', 'peak_distance_CM', 'cluster_sensitivity', 'cluster_precision']].to_csv(os.path.join(output_dir_supplement, 'comparison_scores_matched_reg.csv'))